# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide. 

Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [88]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [90]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('tagsets')
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('omw-1.4')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = list(stopwords.words('english')) 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [91]:
col_names = ["target", "ids", "date", "flag", "user", "text"]

In [92]:
data = pd.read_csv("/content/drive/MyDrive/IronHack/Week_8/archive.zip", 
                   compression = "zip", encoding = "ISO-8859-1", names=col_names)
data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [93]:
# your code here
data.shape #1mln 600 thousands rows!

(1600000, 6)

In [94]:
df = data.sample(20000) # taking 20k rows from the data
df.head()

,target,ids,date,flag,user,text
173873,0,1964229170,Fri May 29 13:55:06 PDT 2009,NO_QUERY,handfulskittles,is so glad the weekend is here. Only one more ...
854590,4,1573421655,Tue Apr 21 01:05:34 PDT 2009,NO_QUERY,josephinelura,@eric_michael no no you're fine ohhh i had a ...
34776,0,1564934493,Mon Apr 20 05:24:32 PDT 2009,NO_QUERY,fearfuldogs,@imsiouxz those puppy millers will keep traine...
1487692,4,2068333935,Sun Jun 07 13:54:22 PDT 2009,NO_QUERY,Luko_S,is still so happy about federer
105042,0,1822825756,Sat May 16 20:28:34 PDT 2009,NO_QUERY,itsAndyD4real,Drinking myself to sleep..baby I miss you


In [95]:
texts = '' # its an empty string and we concat all reviews

for text in df['text']:
    texts += text

corpus = nltk.sent_tokenize(texts) #it removes some special charachters

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [96]:
def clean_up(x):
    """
      
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    
    x = str(x).lower().replace("\\","").replace("_"," ")
    x = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', ' ' , x) 
    x = re.sub(r'\W+',' ',x) # Replace everything non-alpahnumeric by ' '
    x = re.sub(r'\s+',' ',x) # Replace one or more whitespaces by  ' '
    x = re.sub(r'\d+',' ',x) # Replace one or more digits by  ' '
    x = re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)'," ", x) # Replace e-mails by ''
    x = BeautifulSoup(x, 'lxml').get_text().strip()
    # Replace html tags by ''
    x = x.replace(' br ',' ')

    return x

In [97]:
def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    tokens = nltk.word_tokenize(s)
    
    return tokens

In [98]:
def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    ps = PorterStemmer()
    stemmed_output=([ps.stem(w) for w in l])

    lemmatizer = WordNetLemmatizer()
    stemmed_lemmatized_output = ([lemmatizer.lemmatize(w) for w in stemmed_output])
    

    return stemmed_lemmatized_output

In [99]:
def remove_stopwords(l):
  """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
  tokens_without_sw = [word for word in l if not word in stop_words]

    
  return tokens_without_sw

In [100]:
#clean_up() tokenize() stem_and_lemmatize() remove_stopwords()

In [101]:
df['text_processed']=df['text'].apply(clean_up)
df

,target,ids,date,flag,user,text,text_processed
173873,0,1964229170,Fri May 29 13:55:06 PDT 2009,NO_QUERY,handfulskittles,is so glad the weekend is here. Only one more ...,is so glad the weekend is here only one more w...
854590,4,1573421655,Tue Apr 21 01:05:34 PDT 2009,NO_QUERY,josephinelura,@eric_michael no no you're fine ohhh i had a ...,eric michael no no you re fine ohhh i had a dr...
34776,0,1564934493,Mon Apr 20 05:24:32 PDT 2009,NO_QUERY,fearfuldogs,@imsiouxz those puppy millers will keep traine...,imsiouxz those puppy millers will keep trainer...
1487692,4,2068333935,Sun Jun 07 13:54:22 PDT 2009,NO_QUERY,Luko_S,is still so happy about federer,is still so happy about federer
105042,0,1822825756,Sat May 16 20:28:34 PDT 2009,NO_QUERY,itsAndyD4real,Drinking myself to sleep..baby I miss you,drinking myself to sleep baby i miss you
...,...,...,...,...,...,...,...
1577020,4,2189619480,Tue Jun 16 00:28:29 PDT 2009,NO_QUERY,LaRisa_Janai,Happy Born day to Tupac &amp; all the June bab...,happy born day to tupac amp all the june babie...
1000705,4,1880025747,Thu May 21 23:52:07 PDT 2009,NO_QUERY,paul_steele,"@thekeithchegwin Good morning, another happy m...",thekeithchegwin good morning another happy man...
1045685,4,1957575627,Fri May 29 00:56:02 PDT 2009,NO_QUERY,mzweezyfbby,@davidleibrandt cant fucking waaaait till new ...,davidleibrandt cant fucking waaaait till new v...
126075,0,1834443301,Mon May 18 02:57:25 PDT 2009,NO_QUERY,EmelineB,@SooYeonKim bulgogi yes i had some in korea bu...,sooyeonkim bulgogi yes i had some in korea but...


In [102]:
df['text_processed_1']=df['text_processed'].apply(tokenize)

In [103]:
df.head()

,target,ids,date,flag,user,text,text_processed,text_processed_1
173873,0,1964229170,Fri May 29 13:55:06 PDT 2009,NO_QUERY,handfulskittles,is so glad the weekend is here. Only one more ...,is so glad the weekend is here only one more w...,"[is, so, glad, the, weekend, is, here, only, o..."
854590,4,1573421655,Tue Apr 21 01:05:34 PDT 2009,NO_QUERY,josephinelura,@eric_michael no no you're fine ohhh i had a ...,eric michael no no you re fine ohhh i had a dr...,"[eric, michael, no, no, you, re, fine, ohhh, i..."
34776,0,1564934493,Mon Apr 20 05:24:32 PDT 2009,NO_QUERY,fearfuldogs,@imsiouxz those puppy millers will keep traine...,imsiouxz those puppy millers will keep trainer...,"[imsiouxz, those, puppy, millers, will, keep, ..."
1487692,4,2068333935,Sun Jun 07 13:54:22 PDT 2009,NO_QUERY,Luko_S,is still so happy about federer,is still so happy about federer,"[is, still, so, happy, about, federer]"
105042,0,1822825756,Sat May 16 20:28:34 PDT 2009,NO_QUERY,itsAndyD4real,Drinking myself to sleep..baby I miss you,drinking myself to sleep baby i miss you,"[drinking, myself, to, sleep, baby, i, miss, you]"


In [104]:
df['text_processed_2']=df['text_processed_1'].apply(stem_and_lemmatize)

df['text_processed_3']=df['text_processed_2'].apply(remove_stopwords)

In [105]:
df = df.drop(columns=['text_processed', 'text_processed_1', 'text_processed_2'])


In [106]:
df = df.rename(columns={'text_processed_3': 'text_processed'})
df.reset_index()

,index,target,ids,date,flag,user,text,text_processed
0,173873,0,1964229170,Fri May 29 13:55:06 PDT 2009,NO_QUERY,handfulskittles,is so glad the weekend is here. Only one more ...,"[glad, weekend, onli, one, week, left, school,..."
1,854590,4,1573421655,Tue Apr 21 01:05:34 PDT 2009,NO_QUERY,josephinelura,@eric_michael no no you're fine ohhh i had a ...,"[eric, michael, fine, ohhh, dream, dont, remem..."
2,34776,0,1564934493,Mon Apr 20 05:24:32 PDT 2009,NO_QUERY,fearfuldogs,@imsiouxz those puppy millers will keep traine...,"[imsiouxz, puppi, miller, keep, trainer, biz, ..."
3,1487692,4,2068333935,Sun Jun 07 13:54:22 PDT 2009,NO_QUERY,Luko_S,is still so happy about federer,"[still, happi, feder]"
4,105042,0,1822825756,Sat May 16 20:28:34 PDT 2009,NO_QUERY,itsAndyD4real,Drinking myself to sleep..baby I miss you,"[drink, sleep, babi, miss]"
...,...,...,...,...,...,...,...,...
19995,1577020,4,2189619480,Tue Jun 16 00:28:29 PDT 2009,NO_QUERY,LaRisa_Janai,Happy Born day to Tupac &amp; all the June bab...,"[happi, born, day, tupac, amp, june, babi, inc..."
19996,1000705,4,1880025747,Thu May 21 23:52:07 PDT 2009,NO_QUERY,paul_steele,"@thekeithchegwin Good morning, another happy m...","[thekeithchegwin, good, morn, anoth, happi, ma..."
19997,1045685,4,1957575627,Fri May 29 00:56:02 PDT 2009,NO_QUERY,mzweezyfbby,@davidleibrandt cant fucking waaaait till new ...,"[davidleibrandt, cant, fuck, waaaait, till, ne..."
19998,126075,0,1834443301,Mon May 18 02:57:25 PDT 2009,NO_QUERY,EmelineB,@SooYeonKim bulgogi yes i had some in korea bu...,"[sooyeonkim, bulgogi, ye, korea, unfortunatli,..."


In [107]:
# your code here

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [108]:
df['text_processed'][0:5]

173873     [glad, weekend, onli, one, week, left, school,...
854590     [eric, michael, fine, ohhh, dream, dont, remem...
34776      [imsiouxz, puppi, miller, keep, trainer, biz, ...
1487692                                [still, happi, feder]
105042                            [drink, sleep, babi, miss]
Name: text_processed, dtype: object

In [109]:
# your code here
new_list = []

for l in df['text_processed']:
    new_list += l

print(new_list)


['glad', 'weekend', 'onli', 'one', 'week', 'left', 'school', 'kid', 'eric', 'michael', 'fine', 'ohhh', 'dream', 'dont', 'rememb', 'though', 'imsiouxz', 'puppi', 'miller', 'keep', 'trainer', 'biz', 'long', 'time', 'best', 'luck', 'patienc', 'amp', 'chees', 'boy', 'still', 'happi', 'feder', 'drink', 'sleep', 'babi', 'miss', 'awajust', 'oh', 'em', 'gee', 'amaz', 'song', 'take', 'shower', 'eat', 'lunch', 'go', 'shop', 'w', 'mommi', 'amp', 'sister', 'antoinether', 'doubl', 'cheesi', 'beef', 'burrito', 'overstuf', 'burrito', 'reason', 'pack', 'aint', 'togeth', 'anymor', 'lol', 'head', 'nana', 'hous', 'greet', 'fam', 'seattl', 'love', 'new', 'famili', 'great', 'littl', 'sister', 'awesom', 'littl', 'cousin', 'fabul', 'eccentr', 'angel', 'brow', 'digit', 'art', 'tutori', 'like', 'found', 'hen', 'site', 'refresh', 'knowledg', 'skill', 'juryan', 'virginbreakfast', 'good', 'morn', 'guy', 'mikefoden', 'lol', 'uncyherb', 'gold', 'www', 'tweetingtoohard', 'com', 'lilylauren', 'lock', 'account', 'ha',

In [110]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
fdist = FreqDist()
for word in new_list:
    fdist[word.lower()] += 1

In [111]:
fdist

FreqDist({'go': 1765, 'day': 1456, 'get': 1431, 'wa': 1312, 'thi': 1216, 'good': 1166, 'work': 1126, 'like': 1081, 'love': 1012, 'quot': 984, ...})

In [112]:
#5000 words with the highest frequency to build the features.
most_freq=fdist.most_common(5000)

most_freq

[('go', 1765),
 ('day', 1456),
 ('get', 1431),
 ('wa', 1312),
 ('thi', 1216),
 ('good', 1166),
 ('work', 1126),
 ('like', 1081),
 ('love', 1012),
 ('quot', 984),
 ('u', 912),
 ('today', 881),
 ('got', 858),
 ('time', 849),
 ('thank', 777),
 ('lol', 768),
 ('miss', 724),
 ('one', 707),
 ('back', 699),
 ('know', 677),
 ('think', 658),
 ('want', 656),
 ('see', 654),
 ('feel', 640),
 ('amp', 612),
 ('im', 601),
 ('hope', 600),
 ('realli', 591),
 ('night', 576),
 ('still', 559),
 ('need', 556),
 ('watch', 539),
 ('new', 527),
 ('make', 520),
 ('well', 516),
 ('na', 508),
 ('much', 503),
 ('ha', 492),
 ('home', 490),
 ('come', 483),
 ('morn', 474),
 ('look', 470),
 ('oh', 459),
 ('wish', 437),
 ('twitter', 436),
 ('sleep', 427),
 ('tomorrow', 418),
 ('last', 412),
 ('great', 403),
 ('wait', 394),
 ('onli', 382),
 ('right', 380),
 ('haha', 374),
 ('whi', 370),
 ('follow', 370),
 ('bad', 367),
 ('sad', 358),
 ('fun', 353),
 ('thing', 343),
 ('veri', 343),
 ('tri', 339),
 ('would', 333),
 ('wee

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [113]:
# your code here
cols = {word[0]: [] for word in most_freq}
words = pd.DataFrame(cols)

words

,go,day,get,wa,thi,good,work,like,love,quot,...,koko,hairdryer,rascal,flatt,cp,puff,wayyyi,dlsu,jennif,alter


In [114]:
def find_features(document):
  words=list(document)
  features={}
  most_frequent_words = [ tupple[0] for tupple in most_freq ] # most_freq -> list of tupples ( word, freq)
  for word in most_frequent_words:
    features[word] = (word in words)
   
  return features


In [115]:
df['features'] = df['text_processed'].apply(find_features)
df.head()

,target,ids,date,flag,user,text,text_processed,features
173873,0,1964229170,Fri May 29 13:55:06 PDT 2009,NO_QUERY,handfulskittles,is so glad the weekend is here. Only one more ...,"[glad, weekend, onli, one, week, left, school,...","{'go': False, 'day': False, 'get': False, 'wa'..."
854590,4,1573421655,Tue Apr 21 01:05:34 PDT 2009,NO_QUERY,josephinelura,@eric_michael no no you're fine ohhh i had a ...,"[eric, michael, fine, ohhh, dream, dont, remem...","{'go': False, 'day': False, 'get': False, 'wa'..."
34776,0,1564934493,Mon Apr 20 05:24:32 PDT 2009,NO_QUERY,fearfuldogs,@imsiouxz those puppy millers will keep traine...,"[imsiouxz, puppi, miller, keep, trainer, biz, ...","{'go': False, 'day': False, 'get': False, 'wa'..."
1487692,4,2068333935,Sun Jun 07 13:54:22 PDT 2009,NO_QUERY,Luko_S,is still so happy about federer,"[still, happi, feder]","{'go': False, 'day': False, 'get': False, 'wa'..."
105042,0,1822825756,Sat May 16 20:28:34 PDT 2009,NO_QUERY,itsAndyD4real,Drinking myself to sleep..baby I miss you,"[drink, sleep, babi, miss]","{'go': False, 'day': False, 'get': False, 'wa'..."


In [116]:
df.shape

(20000, 8)

In [117]:
len(df)/2000

10.0

Making chunks, as my code collapsed once, not enough RAM

In [118]:
chunks_dfs = []
for i in range(10):
  print("Computing chunk: ",i+1)
  chunks_dfs.append(pd.json_normalize(df[['features']].reset_index(drop=True).iloc[i*2000:(i+1)*2000,0]))
  words = pd.concat(chunks_dfs, axis=0)



Computing chunk:  1
Computing chunk:  2
Computing chunk:  3
Computing chunk:  4
Computing chunk:  5
Computing chunk:  6
Computing chunk:  7
Computing chunk:  8
Computing chunk:  9
Computing chunk:  10


In [119]:
words=words.reset_index()

In [120]:
target=df[['target']]

In [121]:
target

,target
173873,0
854590,4
34776,0
1487692,4
105042,0
...,...
1577020,4
1000705,4
1045685,4
126075,0


In [122]:
target=target[['target']].replace(0,False,regex=True) #0-is a negative tweet, so I make it False as column name will be "is_positive"


In [123]:
target=target[['target']].replace(4,True,regex=True)#4-is a positive tweet, so I make it True.
target

,target
173873,False
854590,True
34776,False
1487692,True
105042,False
...,...
1577020,True
1000705,True
1045685,True
126075,False


In [124]:
target = target.rename(columns={'target': 'is_positive'}) #renaming the column according to the task

In [125]:
target=target.reset_index()
target

,index,is_positive
0,173873,False
1,854590,True
2,34776,False
3,1487692,True
4,105042,False
...,...,...
19995,1577020,True
19996,1000705,True
19997,1045685,True
19998,126075,False


In [126]:
target = target.drop(columns=['index'])
target


,is_positive
0,False
1,True
2,False
3,True
4,False
...,...
19995,True
19996,True
19997,True
19998,False


In [127]:
new=pd.concat([words,target],axis=1)

In [128]:
new

,index,go,day,get,wa,thi,good,work,like,love,...,hairdryer,rascal,flatt,cp,puff,wayyyi,dlsu,jennif,alter,is_positive
0,0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,2,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,4,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1995,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
19996,1996,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
19997,1997,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
19998,1998,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [130]:
new = new.drop(columns=['index'])
new


,go,day,get,wa,thi,good,work,like,love,quot,...,hairdryer,rascal,flatt,cp,puff,wayyyi,dlsu,jennif,alter,is_positive
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
19996,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
19997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
19998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [131]:
df.reset_index(drop=True,inplace=True)
df.head()

,target,ids,date,flag,user,text,text_processed,features
0,0,1964229170,Fri May 29 13:55:06 PDT 2009,NO_QUERY,handfulskittles,is so glad the weekend is here. Only one more ...,"[glad, weekend, onli, one, week, left, school,...","{'go': False, 'day': False, 'get': False, 'wa'..."
1,4,1573421655,Tue Apr 21 01:05:34 PDT 2009,NO_QUERY,josephinelura,@eric_michael no no you're fine ohhh i had a ...,"[eric, michael, fine, ohhh, dream, dont, remem...","{'go': False, 'day': False, 'get': False, 'wa'..."
2,0,1564934493,Mon Apr 20 05:24:32 PDT 2009,NO_QUERY,fearfuldogs,@imsiouxz those puppy millers will keep traine...,"[imsiouxz, puppi, miller, keep, trainer, biz, ...","{'go': False, 'day': False, 'get': False, 'wa'..."
3,4,2068333935,Sun Jun 07 13:54:22 PDT 2009,NO_QUERY,Luko_S,is still so happy about federer,"[still, happi, feder]","{'go': False, 'day': False, 'get': False, 'wa'..."
4,0,1822825756,Sat May 16 20:28:34 PDT 2009,NO_QUERY,itsAndyD4real,Drinking myself to sleep..baby I miss you,"[drink, sleep, babi, miss]","{'go': False, 'day': False, 'get': False, 'wa'..."


In 'df' dataframe last column is a dictionary of words that we need
in. 'new' dataframe last column is a column with the flags True -False.
We make a zip and after take first element - item[0] -dictionary of words
and item[1] is a true-false label from target column 

In [139]:
# [([],) for item in ]
the_list = []
for i in range(10):
  the_list += [(item[0],item[1]) for item in zip(list(df.iloc[i*2000:(i+1)*2000,-1]),list(new.iloc[i*2000:(i+1)*2000,-1]))]
  
the_list[0]

({'go': False,
  'day': False,
  'get': False,
  'wa': False,
  'thi': False,
  'good': False,
  'work': False,
  'like': False,
  'love': False,
  'quot': False,
  'u': False,
  'today': False,
  'got': False,
  'time': False,
  'thank': False,
  'lol': False,
  'miss': False,
  'one': True,
  'back': False,
  'know': False,
  'think': False,
  'want': False,
  'see': False,
  'feel': False,
  'amp': False,
  'im': False,
  'hope': False,
  'realli': False,
  'night': False,
  'still': False,
  'need': False,
  'watch': False,
  'new': False,
  'make': False,
  'well': False,
  'na': False,
  'much': False,
  'ha': False,
  'home': False,
  'come': False,
  'morn': False,
  'look': False,
  'oh': False,
  'wish': False,
  'twitter': False,
  'sleep': False,
  'tomorrow': False,
  'last': False,
  'great': False,
  'wait': False,
  'onli': True,
  'right': False,
  'haha': False,
  'whi': False,
  'follow': False,
  'bad': False,
  'sad': False,
  'fun': False,
  'thing': False,
  'ver

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [147]:
# your code here
#Because of having the problems with memory, I have taken first 1000 tweets for training and next 1000 for testing
training_set=the_list[:1000]
testing_set=the_list[1000:2000]


In [151]:
classifier=nltk.NaiveBayesClassifier.train(training_set)


In [153]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)
#the accuracy is ok, according to video it can be even smaller.

Classifier accuracy percent: 67.10000000000001


In [146]:
classifier.show_most_informative_features(15)
#this is quiet interesting, it's the most 15 informative features. here we can see that word "sad" is 16.6 more often met in negative tweet.
#"welcome" - 15.3 more often in positive tweet, "damn" - 12.5 times more often in negative tweet and etc.

Most Informative Features
                     sad = True            False : True   =     16.6 : 1.0
                  welcom = True             True : False  =     15.3 : 1.0
                    damn = True            False : True   =     12.5 : 1.0
                     cri = True            False : True   =     12.2 : 1.0
                    hehe = True             True : False  =     12.1 : 1.0
                   sadli = True            False : True   =     11.7 : 1.0
                    sigh = True            False : True   =     11.7 : 1.0
                    film = True             True : False  =     11.6 : 1.0
                     ugh = True            False : True   =     11.2 : 1.0
                 batteri = True            False : True   =      9.7 : 1.0
                 fantast = True             True : False  =      9.6 : 1.0
                    pain = True            False : True   =      9.3 : 1.0
                     ear = True            False : True   =      9.1 : 1.0

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
#done above

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here